In [4]:
# For Data Analysis
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# For Modelling and Evaluating Performance of the Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Lasso, LinearRegression, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import LabelEncoder

In [5]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", lambda x: "%.2f" %x)
pd.set_option("display.width", 500)

In [6]:
df_ = pd.read_csv("Melbourne_housing_FULL.csv")
df = df_.copy()

### Variable Definitions
* **Suburb** : Suburb
* **Address**: Address
* **Rooms**: Number of rooms
* **Price**: Price in Australian dollars
* **Method**:
  * **S**: Property sold
  * **SP**: Property sold prior
  * **PI**: Property passed in
  * **PN**: Sold prior not disclosed
  * **SN**: Sold not disclosed
  * **NB**: No bid
  * **VB**: Vendor bid
  * **W**: Withdrawn prior to auction
  * **SA**: Sold after auction
  * **SS**: Sold after auction price not disclosed
  * **N/A**: Price or highest bid not available
* **Type**:
  * **br**: Bedroom(s)
  * **h**: House, cottage, villa, semi, terrace
  * **u**: unit, duplex
  * **t**: townhouse
  * **dev** site: Development site
  * **o res**: Other residential
* **SellerG**: Real Estate Agent
* **Date**: Date sold
* **Distance**: Distance from CBD in kilometers
* **Regionname**: General Region (West, North West, North, North east …etc)
* **Propertycount**: Number of properties that exist in the suburb.
* **Bedroom2** : Scraped # of Bedrooms (from different source)
* **Bathroom**: Number of Bathrooms
* **Car**: Number of carspots
* **Landsize**: Land Size in Metres
* **BuildingArea**: Building Size in Metres
* **YearBuilt**: Year the house was built
* **CouncilArea**: Governing council for the area
* **Lattitude**: Self explanitory
* **Longtitude**: Self explanitory

In [7]:
# Check first 5 rows
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2.00,h,NaN,SS,Jellis,3/09/2016,2.50,3067.00,2.00,1.00,1.00,126.00,NaN,NaN,Yarra City Council,-37.80,145.00,Northern Metropolitan,4019.00
1,Abbotsford,85 Turner St,2.00,h,1480000.00,S,Biggin,3/12/2016,2.50,3067.00,2.00,1.00,1.00,202.00,NaN,NaN,Yarra City Council,-37.80,145.00,Northern Metropolitan,4019.00
2,Abbotsford,25 Bloomburg St,2.00,h,1035000.00,S,Biggin,4/02/2016,2.50,3067.00,2.00,1.00,0.00,156.00,79.00,1900.00,Yarra City Council,-37.81,144.99,Northern Metropolitan,4019.00
3,Abbotsford,18/659 Victoria St,3.00,u,NaN,VB,Rounds,4/02/2016,2.50,3067.00,3.00,2.00,1.00,0.00,NaN,NaN,Yarra City Council,-37.81,145.01,Northern Metropolitan,4019.00
4,Abbotsford,5 Charles St,3.00,h,1465000.00,SP,Biggin,4/03/2017,2.50,3067.00,3.00,2.00,0.00,134.00,150.00,1900.00,Yarra City Council,-37.81,144.99,Northern Metropolitan,4019.00


In [8]:
# Check last 5 rows
df.tail()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
7260,North Melbourne,11/368 Dryburgh St,1.00,u,358000.00,SP,Jellis,27/11/2016,2.30,3051.00,1.00,1.00,1.00,649.00,43.00,1970.00,Melbourne City Council,-37.80,144.94,Northern Metropolitan,6821.00
7261,North Melbourne,103/171 Flemington Rd,1.00,u,320000.00,S,W.B.,27/11/2016,2.30,3051.00,1.00,1.00,1.00,0.00,45.00,1975.00,Melbourne City Council,-37.80,144.95,Northern Metropolitan,6821.00
7262,North Melbourne,10 Munster Tce,3.00,t,1000000.00,VB,Nelson,27/11/2016,2.30,3051.00,3.00,2.00,1.00,61.00,170.00,1920.00,Melbourne City Council,-37.80,144.94,Northern Metropolitan,6821.00
7263,North Melbourne,203/150 Peel St,2.00,u,598000.00,S,Alexkarbon,27/11/2016,2.30,3051.00,2.00,1.00,1.00,0.00,85.00,2005.00,Melbourne City Council,-37.80,144.96,Northern Metropolitan,6821.00
7264,North Melbourne,2/101 Leveson St,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Check total observations and columns
df.shape

(7265, 21)

In [10]:
# Information about DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7265 entries, 0 to 7264
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         7265 non-null   object 
 1   Address        7265 non-null   object 
 2   Rooms          7264 non-null   float64
 3   Type           7264 non-null   object 
 4   Price          5590 non-null   float64
 5   Method         7264 non-null   object 
 6   SellerG        7264 non-null   object 
 7   Date           7264 non-null   object 
 8   Distance       7264 non-null   float64
 9   Postcode       7264 non-null   float64
 10  Bedroom2       5693 non-null   float64
 11  Bathroom       5693 non-null   float64
 12  Car            5693 non-null   float64
 13  Landsize       5687 non-null   float64
 14  BuildingArea   3291 non-null   float64
 15  YearBuilt      3738 non-null   float64
 16  CouncilArea    7264 non-null   object 
 17  Lattitude      5719 non-null   float64
 18  Longtitu

In [11]:
# Check missing values
df.isnull().sum()

Suburb              0
Address             0
Rooms               1
Type                1
Price            1675
Method              1
SellerG             1
Date                1
Distance            1
Postcode            1
Bedroom2         1572
Bathroom         1572
Car              1572
Landsize         1578
BuildingArea     3974
YearBuilt        3527
CouncilArea         1
Lattitude        1546
Longtitude       1546
Regionname          1
Propertycount       1
dtype: int64

In [12]:
# Check descriptive statistics
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Rooms,7264.00,3.00,1.00,1.00,2.00,3.00,4.00,10.00
Price,5590.00,1153092.34,704071.01,85000.00,675000.00,953000.00,1440000.00,6500000.00
Distance,7264.00,8.99,3.33,1.60,6.40,9.20,11.40,15.00
Postcode,7264.00,3102.13,58.72,3000.00,3046.00,3103.00,3147.00,3206.00
Bedroom2,5693.00,2.93,1.04,0.00,2.00,3.00,3.00,30.00
Bathroom,5693.00,1.60,0.76,0.00,1.00,1.00,2.00,12.00
Car,5693.00,1.61,0.98,0.00,1.00,2.00,2.00,26.00
Landsize,5687.00,463.60,1228.22,0.00,158.00,379.00,645.00,75100.00
BuildingArea,3291.00,157.92,170.54,0.00,93.00,131.00,190.00,6791.00
YearBuilt,3738.00,1959.26,37.81,1830.00,1930.00,1960.00,1996.00,2106.00


In [13]:
duplicate_rows = df[df.duplicated()]

In [14]:
df = df.drop_duplicates()

In [15]:
df.shape

(7265, 21)

In [16]:
df.dropna(subset=['Price'], inplace=True)

In [17]:
df.shape

(5590, 21)

In [18]:
# Unique values for all columns
for col in df.columns:
  print(f"{col} have {df[col].nunique()} unique values.\n")

Suburb have 76 unique values.

Address have 5541 unique values.

Rooms have 9 unique values.

Type have 3 unique values.

Price have 1494 unique values.

Method have 5 unique values.

SellerG have 170 unique values.

Date have 35 unique values.

Distance have 60 unique values.

Postcode have 63 unique values.

Bedroom2 have 10 unique values.

Bathroom have 8 unique values.

Car have 9 unique values.

Landsize have 994 unique values.

BuildingArea have 380 unique values.

YearBuilt have 120 unique values.

CouncilArea have 18 unique values.

Lattitude have 1894 unique values.

Longtitude have 2165 unique values.

Regionname have 4 unique values.

Propertycount have 76 unique values.



In [19]:
# Catching categorical columns

def grab_col_names(dataframe, cat_th=10, car_th=20):
    """
    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
            Degisken isimleri alinmak istenilen dataframe

        cat_th: int, optional
            Numerik fakat kategorik olan degiskenler için sinif esik degeri

        car_th: int, optional
            Kategorik fakat kardinal degiskenler için sinif esik degeri

    Returns
    ------
        cat_cols: list
            Kategorik degisken listesi

        num_cols: list
            Numerik degisken listesi


    """
    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                  dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                  dataframe[col].dtypes =="O"]

    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]

    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f"Categorical Columns: {len(cat_cols)}")
    print(f"Numerical Columns: {len(num_cols)}")
    print(f"Categorical but Cardinal Columns: {len(cat_but_car)}")
    print(f"Numerical but Categorical Columns: {len(num_but_cat)}")

    return cat_cols, num_cols, cat_but_car

In [20]:
cat_cols, num_cols, cat_but_car = grab_col_names(df, car_th=20)

Observations: 5590
Variables: 21
Categorical Columns: 7
Numerical Columns: 10
Categorical but Cardinal Columns: 4
Numerical but Categorical Columns: 3


In [21]:
for col in cat_cols:
    print(f"{col} - have {df[col].nunique()} unique values")

Type - have 3 unique values
Method - have 5 unique values
CouncilArea - have 18 unique values
Regionname - have 4 unique values
Rooms - have 9 unique values
Bathroom - have 8 unique values
Car - have 9 unique values


In [22]:
for col in num_cols:
    print(f"{col} - have {df[col].nunique()} unique values")

Price - have 1494 unique values
Distance - have 60 unique values
Postcode - have 63 unique values
Bedroom2 - have 10 unique values
Landsize - have 994 unique values
BuildingArea - have 380 unique values
YearBuilt - have 120 unique values
Lattitude - have 1894 unique values
Longtitude - have 2165 unique values
Propertycount - have 76 unique values


In [23]:
for col in cat_but_car:
    print(f"{col} - have {df[col].nunique()} unique values")

Suburb - have 76 unique values
Address - have 5541 unique values
SellerG - have 170 unique values
Date - have 35 unique values


In [24]:
# Seperating train-test

X = df.drop("Price", axis=1)
y = df["Price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True)

In [25]:
X_train.shape

(4192, 20)

In [26]:
X_test.shape

(1398, 20)

In [27]:
y_train.shape[0]

4192

In [28]:
y_test.shape[0]

1398

In [29]:
# Definin outlier thresholds

def outlier_thresholds(dataframe, col_name, q1=0.05, q3=0.95):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)

    interquantile_range = quartile3 - quartile1

    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range

    return low_limit, up_limit

In [30]:
# Check outlier for a variable

def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)

    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False

In [31]:
num_cols = [col for col in num_cols if col not in "Price"]

In [32]:
for col in num_cols:
    low_limit, up_limit = outlier_thresholds(X_train, col)

    print(f"{col}\nLow Limit = {low_limit}\nUp Limit = {up_limit}\n")

Distance
Low Limit = -13.299999999999988
Up Limit = 30.21999999999999

Postcode
Low Limit = 2755.5
Up Limit = 3447.5

Bedroom2
Low Limit = -1.0
Up Limit = 7.0

Landsize
Low Limit = -1373.25
Up Limit = 2288.75

BuildingArea
Low Limit = -329.32499999999993
Up Limit = 692.8749999999999

YearBuilt
Low Limit = 1707.0
Up Limit = 2195.0

Lattitude
Low Limit = -38.22640000000001
Up Limit = -37.4324

Longtitude
Low Limit = 144.51017500000003
Up Limit = 145.45877499999997

Propertycount
Low Limit = -10574.5
Up Limit = 24437.5



In [33]:
X_train_outlier = []

for col in num_cols:
    print(col, check_outlier(X_train, col))

    if check_outlier(X_train, col) == True:
        X_train_outlier.append(col)

Distance False
Postcode False
Bedroom2 True
Landsize True
BuildingArea True
YearBuilt False
Lattitude False
Longtitude False
Propertycount False


In [34]:
X_train_outlier

['Bedroom2', 'Landsize', 'BuildingArea']

There are some outliers in **Rooms**, **Postcode**, **Bedroom2**, **Bathroom**, **Car**, **Landsize**, **BuildingArea**, and **YearBuilt** variables. **Price** is our target value. So we check the ourliers without Price variable.

In [35]:
# Catching outliers in variables

def grab_outliers(dataframe, col_name, index=False):
    low, up = outlier_thresholds(dataframe, col_name)

    if dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))].shape[0] > 10:
        print(dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))].head())
    else:
        print(dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))])

    if index:
        outlier_index = dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))].index
        return outlier_index

In [36]:
outlier_indices = []

for col in X_train_outlier:
    indices = grab_outliers(X_train, col, index=True)
    outlier_indices.extend(indices)

        Suburb         Address  Rooms Type Method        SellerG        Date  Distance  Postcode  Bedroom2  Bathroom  Car  Landsize  BuildingArea  YearBuilt              CouncilArea  Lattitude  Longtitude             Regionname  Propertycount
616  Ashburton  115 Ashburn Gr   8.00    h      S  hockingstuart  10/09/2016     11.00   3147.00      9.00      7.00 4.00   1472.00        618.00    2009.00  Boroondara City Council     -37.87      145.08  Southern Metropolitan        3052.00
             Suburb          Address  Rooms Type Method   SellerG        Date  Distance  Postcode  Bedroom2  Bathroom  Car  Landsize  BuildingArea  YearBuilt             CouncilArea  Lattitude  Longtitude             Regionname  Propertycount
5797     Kensington  28/18 Mawbey St   3.00    u      S  Woodards  10/12/2016      4.20   3031.00      3.00      1.00 1.00   4440.00           NaN        NaN  Melbourne City Council     -37.79      144.92  Northern Metropolitan        5263.00
4256  Fitzroy North   186 Qu

In [37]:
len(outlier_indices)

39

In [38]:
X_train.shape

(4192, 20)

In [39]:
X_train = X_train.drop(index=outlier_indices)

In [40]:
X_train.shape[0]

4154

In [41]:
y_train.shape[0]

4192

In [42]:
y_train = y_train.drop(index=outlier_indices)

In [43]:
y_train.shape[0]

4154

In [44]:
20435 - 19899

536

We removed totally 536 values from X_train dataset.

In [45]:
num_cols

['Distance',
 'Postcode',
 'Bedroom2',
 'Landsize',
 'BuildingArea',
 'YearBuilt',
 'Lattitude',
 'Longtitude',
 'Propertycount']

In [46]:
label_encoder = LabelEncoder()

In [47]:
for col in cat_cols:
    X_train[col] = label_encoder.fit_transform(X_train[col])

In [48]:
for col in cat_but_car:
    X_train[col] = label_encoder.fit_transform(X_train[col])

In [49]:
imputer = KNNImputer(n_neighbors=5)

X_train_imputed = imputer.fit_transform(X_train)

In [50]:
for col in cat_cols:
    X_test[col] = label_encoder.fit_transform(X_test[col])

In [51]:
for col in cat_but_car:
    X_test[col] = label_encoder.fit_transform(X_test[col])

In [52]:
imputer = KNNImputer(n_neighbors=5)

X_test_imputed = imputer.fit_transform(X_test)

In [53]:
missing_values = np.isnan(X_train_imputed)
missing_counts = np.sum(missing_values, axis=0)
total_missing = np.sum(missing_counts)

In [54]:
print("Missing Values for all variables:")
print(missing_counts)

Missing Values for all variables:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [55]:
X_train_imputed.shape[0]

4154

In [56]:
y_train.shape[0]

4154

In [57]:
estimator = DecisionTreeRegressor()

models = {
    'Lasso': Lasso(),
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'ElasticNet': ElasticNet(),
    'KNeighborsRegressor': KNeighborsRegressor(),
    'RandomForestRegressor': RandomForestRegressor(),
    'GradientBoostingRegressor': GradientBoostingRegressor(),
    'AdaBoostRegressor': AdaBoostRegressor(n_estimators = 20, learning_rate = 1, loss = 'exponential', random_state = 42, estimator=estimator)
}

In [58]:
for model_name, model in models.items():
    model.fit(X_train_imputed, y_train)
    predict = model.predict(X_test_imputed)
    mae = mean_absolute_error(y_test, predict)
    mse = mean_squared_error(y_test, predict)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predict)

    print(f"##########{model_name}##########")
    print("Mean Absolute Error:  ",mae)
    print("Mean Seqaure Error:  ",mse)
    print("R Mean Seqaure Error:  ",rmse)
    print("R2:  ",r2)
    print("Model Skoru:  ",model.score(X_test_imputed,y_test))
    print("\n")

##########Lasso##########
Mean Absolute Error:   347443.68331809534
Mean Seqaure Error:   381255976212.63226
R Mean Seqaure Error:   617459.2911379925
R2:   0.26212007229577217
Model Skoru:   0.26212007229577217


##########LinearRegression##########
Mean Absolute Error:   347446.12776020425
Mean Seqaure Error:   381252413692.098
R Mean Seqaure Error:   617456.4063090592
R2:   0.2621269671710219
Model Skoru:   0.2621269671710219


##########Ridge##########
Mean Absolute Error:   347426.9620866628
Mean Seqaure Error:   383754307715.2416
R Mean Seqaure Error:   619479.0615632151
R2:   0.25728482043470113
Model Skoru:   0.25728482043470113


##########ElasticNet##########
Mean Absolute Error:   361527.81361492584
Mean Seqaure Error:   514284720312.2902
R Mean Seqaure Error:   717136.4725854419
R2:   0.004657248882625198
Model Skoru:   0.004657248882625198


##########KNeighborsRegressor##########
Mean Absolute Error:   424551.58855507866
Mean Seqaure Error:   405949908188.9147
R Mean Seqa